### Problem 84: Monopoly Odds
<p>In the game, <strong>Monopoly</strong>, the standard board is set up in the following way:</p>
<div class="center">
<img src="https://projecteuler.net/resources/images/0084_monopoly_board.png?1678992052" alt="0084_monopoly_board.png">
</div>
<p>A player starts on the GO square and adds the scores on two 6-sided dice to determine the number of squares they advance in a clockwise direction. Without any further rules we would expect to visit each square with equal probability: 2.5%. However, landing on G2J (Go To Jail), CC (community chest), and CH (chance) changes this distribution.</p>
<p>In addition to G2J, and one card from each of CC and CH, that orders the player to go directly to jail, if a player rolls three consecutive doubles, they do not advance the result of their 3rd roll. Instead they proceed directly to jail.</p>
<p>At the beginning of the game, the CC and CH cards are shuffled. When a player lands on CC or CH they take a card from the top of the respective pile and, after following the instructions, it is returned to the bottom of the pile. There are sixteen cards in each pile, but for the purpose of this problem we are only concerned with cards that order a movement; any instruction not concerned with movement will be ignored and the player will remain on the CC/CH square.</p>
<ul><li>Community Chest (2/16 cards):
<ol><li>Advance to GO</li>
<li>Go to JAIL</li>
</ol></li>
<li>Chance (10/16 cards):
<ol><li>Advance to GO</li>
<li>Go to JAIL</li>
<li>Go to C1</li>
<li>Go to E3</li>
<li>Go to H2</li>
<li>Go to R1</li>
<li>Go to next R (railway company)</li>
<li>Go to next R</li>
<li>Go to next U (utility company)</li>
<li>Go back 3 squares.</li>
</ol></li>
</ul><p>The heart of this problem concerns the likelihood of visiting a particular square. That is, the probability of finishing at that square after a roll. For this reason it should be clear that, with the exception of G2J for which the probability of finishing on it is zero, the CH squares will have the lowest probabilities, as 5/8 request a movement to another square, and it is the final square that the player finishes at on each roll that we are interested in. We shall make no distinction between "Just Visiting" and being sent to JAIL, and we shall also ignore the rule about requiring a double to "get out of jail", assuming that they pay to get out on their next turn.</p>
<p>By starting at GO and numbering the squares sequentially from 00 to 39 we can concatenate these two-digit numbers to produce strings that correspond with sets of squares.</p>
<p>Statistically it can be shown that the three most popular squares, in order, are JAIL (6.24%) = Square 10, E3 (3.18%) = Square 24, and GO (3.09%) = Square 00. So these three most popular squares can be listed with the six-digit modal string: 102400.</p>
<p>If, instead of using two 6-sided dice, two 4-sided dice are used, find the six-digit modal string.</p>


In [250]:
import numpy as np
import math
import collections
import random

In [291]:
monopoly_board = {0:['GO','GO']
                  ,1:['Reg','A1']
                  ,2:['CC','CC1']
                  ,3:['Reg','A2']
                  ,4:['Reg','T1']
                  ,5:['R','R1']
                  ,6:['Reg','B1']
                  ,7:['CH','CH1']
                  ,8:['Reg','B2']
                  ,9:['Reg','B3']
                  ,10:['J','Jail']
                  ,11:['Reg','C1']
                  ,12:['U','U1']
                  ,13:['Reg','C2']
                  ,14:['Reg','C3']
                  ,15:['R','R2']
                  ,16:['Reg','D1']
                  ,17:['CC','CC2']
                  ,18:['Reg','D2']
                  ,19:['Reg','D3']
                  ,20:['Reg','FP']
                  ,21:['Reg','E1']
                  ,22:['CH','CH2']
                  ,23:['Reg','E2']
                  ,24:['Reg','E3']
                  ,25:['R','R3']
                  ,26:['Reg','F1']
                  ,27:['Reg','F2']
                  ,28:['U','U2']
                  ,29:['Reg','F3']
                  ,30:['G2J','G2J']
                  ,31:['Reg','G1']
                  ,32:['Reg','G2']
                  ,33:['CC','CC3']
                  ,34:['Reg','G3']
                  ,35:['R','R4']
                  ,36:['CH','CH3']
                  ,37:['Reg','H1']
                  ,38:['Reg','T2']
                  ,39:['Reg','H2']};

In [540]:
def roll_2_dice(n=4):
    die_1 = random.randint(1,n)
    die_2 = random.randint(1,n)
    return die_1, die_2

In [541]:
def community_card(squareID):
    cardNum = random.randint(1,16)
        
    if cardNum == 1:
        return 0
    
    if cardNum == 2:
        return 10
    
    else:
        return squareID

In [542]:
def chance_card(squareID):
    cardNum = random.randint(1,16)
    
    if cardNum > 10: # Don't move
        return squareID
    
    if cardNum == 1: # Back 3
        return squareID-3
    
    if cardNum in [2,3]: # Next railway
        if squareID == 7:
            return 15
        elif squareID == 22:
            return 25
        elif squareID == 36:
            return 5
        
    if cardNum in [4]: # Next utility
        if squareID == 7:
            return 12
        elif squareID == 22:
            return 28
        elif squareID == 36:
            return 12
    
    else: # Other moves
        return random.choice([0, 5, 10, 11, 24, 39])

In [543]:
def spot_action(squareNum):
    global double_count
    
    squareType = monopoly_board[squareNum][0]
    
    if squareType not in ['CC','G2J','CH']: 
        return squareNum
    if squareType in ['G2J']:
        double_count[-1] = 0
        return 10
    
    if squareType in ['CC']:
        new_square = community_card(squareNum)
        if  new_square == 10:  
            double_count[-1] = 0
        return new_square
    
    if squareType in ['CH']:
        new_square = chance_card(squareNum)
        if  new_square == 10: double_count[-1] = 0
        return new_square

In [544]:
def perform_turn(current_square):
    global dice_hist, square_hist, double_count
    square_hist.append(current_square)
    dice = roll_2_dice()
    dice_hist.append(dice)
    if dice[0]==dice[1]:
        double_count.extend([double_count[-1]+1]) 
        if double_count[-1] >=3:
            double_count.extend([0])
            return 10
        
    else:
        double_count.extend([0])
        
    movement = sum(dice)
    new_square = (current_square + movement)%40
    current_square = spot_action(new_square)
    return current_square

In [545]:
def simulate_game(moves=1000):
    square_hist = []
    dice_hist = []
    double_count = [0]
    current_square = 0
    for i in range(moves):
        current_square = perform_turn(current_square)
        
    return

In [547]:
total_square_hist = []
for i in range(1):
    square_hist = []; dice_hist = []; double_count = [0]
    simulate_game(moves=10_000_000)
    total_square_hist.extend(square_hist)

In [548]:
square_dist = collections.defaultdict(int)
for x in square_hist:
    square_dist[x] += (1/len(square_hist))

square_dist = collections.OrderedDict(sorted(square_dist.items(), key = lambda x: x[1],reverse=True))

In [549]:
for key,val in square_dist.items():
    print(f'Space: {monopoly_board[key][1]}({key}) with chance {100*val:.2f}%')

Space: Jail(10) with chance 6.95%
Space: R2(15) with chance 3.60%
Space: E3(24) with chance 3.29%
Space: D1(16) with chance 3.23%
Space: R3(25) with chance 3.11%
Space: D3(19) with chance 3.06%
Space: E1(21) with chance 3.05%
Space: FP(20) with chance 3.00%
Space: E2(23) with chance 3.00%
Space: D2(18) with chance 2.98%
Space: R1(5) with chance 2.91%
Space: C3(14) with chance 2.88%
Space: U2(28) with chance 2.86%
Space: G1(31) with chance 2.81%
Space: GO(0) with chance 2.79%
Space: F3(29) with chance 2.76%
Space: CC2(17) with chance 2.71%
Space: F1(26) with chance 2.66%
Space: G2(32) with chance 2.59%
Space: F2(27) with chance 2.55%
Space: C2(13) with chance 2.51%
Space: H2(39) with chance 2.48%
Space: C1(11) with chance 2.45%
Space: U1(12) with chance 2.43%
Space: T1(4) with chance 2.27%
Space: CC3(33) with chance 2.20%
Space: G3(34) with chance 2.20%
Space: H1(37) with chance 2.12%
Space: B2(8) with chance 2.11%
Space: B3(9) with chance 2.10%
Space: B1(6) with chance 2.09%
Space: T2(